In [1]:
!pip install PyMuPDF -q
!pip install gmft -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [2]:
import fitz  # PyMuPDF
from gmft.auto import AutoTableDetector, AutoTableFormatter
from gmft.pdf_bindings import PyPDFium2Document
import os
from tqdm import tqdm
import pandas as pd

DPI = 150
detector = AutoTableDetector()
formatter = AutoTableFormatter()





/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/273 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

In [21]:
def is_page_searchable(page, min_chars=10):
    text = page.get_text().strip()
    return len(text) >= min_chars


def has_tables(pdf_path, page_number):
    doc = PyPDFium2Document(pdf_path)
    try:
        page = doc[page_number]
        table_regions = list(detector.extract(page))
        return len(table_regions) > 0
    finally:
        doc.close()


def ocr_page(page, page_number):
    pix = page.get_pixmap(dpi=DPI)
    imgpdf = fitz.open("pdf", pix.pdfocr_tobytes())
    temp_path = f"temp_page_{page_number + 1}.pdf"
    imgpdf.save(temp_path)
    imgpdf.close()
    return temp_path


def extract_tables_from_page(pdf_path, page_number):
    doc = PyPDFium2Document(pdf_path)
    page_tables = []
    table_boxes = []
    try:
        page = doc[page_number]
        for cropped in detector.extract(page):
            table_boxes.append(cropped.bbox)
            formatted = formatter.extract(cropped, margin='auto', padding=None)
            df = formatted.df()
            page_tables.append((cropped.bbox, df))
    finally:
        doc.close()
    return page_tables


def smart_pdf_processing_with_text_and_tables(input_pdf, ocr_output_pdf):
    src = fitz.open(input_pdf)
    ocr_doc = fitz.open()
    page_chunks = []

    for i, page in enumerate(src, start=0):
        print(f"\n📄 Page {i + 1}")

        searchable = is_page_searchable(page)
        table_exists = has_tables(input_pdf, i)

        print(f"   🧠 Searchable: {'✅ Yes' if searchable else '❌ No'}")
        print(f"   📊 Tables: {'✅ Yes' if table_exists else '❌ No'}")

        if not table_exists and not searchable:
            continue

        if not searchable:
            temp_path = ocr_page(page, i)
            temp_doc = fitz.open(temp_path)
            ocr_page_obj = temp_doc[0]
            page_tables = extract_tables_from_page(temp_path, 0)
            table_boxes = [box for box, _ in page_tables]
            blocks = ocr_page_obj.get_text("blocks")
            ocr_doc.insert_pdf(temp_doc)
            temp_doc.close()
            os.remove(temp_path)
        else:
            page_tables = extract_tables_from_page(input_pdf, i)
            table_boxes = [box for box, _ in page_tables]
            blocks = page.get_text("blocks")

        # ⬇️ Combine text and tables in top-to-bottom order
        page_output = f"\n--- Page {i + 1} ---\n\n"
        mixed_elements = []

        # Text blocks
        for block in blocks:
            x0, y0, x1, y1, text = block[:5]
            block_rect = fitz.Rect(x0, y0, x1, y1)
            is_table_overlap = any(fitz.Rect(*box).intersects(block_rect) for box in table_boxes)
            if not is_table_overlap and text.strip():
                mixed_elements.append((y0, "text", text.strip()))

        # Table blocks
        for j, (bbox, df) in enumerate(page_tables, 1):
            y_top = bbox[1]
            table_md = f"[Table Page {i+1}.{j}]\n" + df.to_markdown(index=False)
            mixed_elements.append((y_top, "table", table_md))

        # Sort and merge
        mixed_elements.sort(key=lambda x: x[0])
        for _, kind, content in mixed_elements:
            page_output += content.strip() + "\n\n"

        page_chunks.append(page_output)

    if len(ocr_doc) > 0:
        ocr_doc.save(ocr_output_pdf)
        print(f"\n💾 OCR-enhanced pages saved to: {ocr_output_pdf}")
    ocr_doc.close()
    src.close()

    # Save combined document
    with open("document_output.md", "w", encoding="utf-8") as f:
        f.write("\n".join(page_chunks))
    print("\n📄 Combined document saved as: document_output.md")

    return page_chunks


# === Run the processing ===
input_pdf = "/content/Statement of Financial Performance Template.pdf"
ocr_output_pdf = "input-ocr.pdf"

page_chunks = smart_pdf_processing_with_text_and_tables(input_pdf, ocr_output_pdf)

# Example: print the output of Page 1
print("\n✅ Output of Page 1:\n")
print(page_chunks[0])


📄 Page 1
   🧠 Searchable: ✅ Yes
   📊 Tables: ✅ Yes

📄 Page 2
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 3
   🧠 Searchable: ✅ Yes
   📊 Tables: ✅ Yes

📄 Page 4
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 5
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 6
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 7
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 8
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 9
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 10
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 11
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 12
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 13
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 14
   🧠 Searchable: ✅ Yes
   📊 Tables: ✅ Yes

📄 Page 15
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 16
   🧠 Searchable: ✅ Yes
   📊 Tables: ✅ Yes

📄 Page 17
   🧠 Searchable: ✅ Yes
   📊 Tables: ✅ Yes

📄 Page 18
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 19
   🧠 Searchable: ✅ Yes
   📊 Tables: ❌ No

📄 Page 20
   🧠 

In [23]:
print(page_chunks[2])


--- Page 3 ---

Statement of changes in net assets

for the year ended as at 31 March 2008

[Table Page 3.1]
|                                          | Surplus \nfunds R’000   | Capital \nfunds R’000   | Deferred \nincome R’000   | Total R’000   |
|:-----------------------------------------|:------------------------|:------------------------|:--------------------------|:--------------|
| Balance at 1 April 2006                  | 10 225                  | 75 050                  | 4 331                     | 89 606        |
| Net surplus for the year                 | 5 011                   | -                       | -                         | 5 011         |
| Portion of Parliamentary                 |                         |                         |                           |               |
| grants utilised to acquire               |                         |                         |                           |               |
| depreciable fixed assets                 |

In [7]:
!pip install faiss-cpu sentence-transformers mistralai
!pip install mistralai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 25.2 MB/s eta 0:00:00


In [24]:
import os
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from mistralai import Mistral

# ✅ STEP 1: Embed chunks from your PDF output (page_chunks)
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

print("\n🔍 Embedding document chunks...")
embeddings = embed_model.encode(page_chunks, convert_to_numpy=True)

# ✅ STEP 2: Create FAISS index
dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings)

# ✅ STEP 3: Store mapping of index to chunk text
index_to_chunk = {i: chunk for i, chunk in enumerate(page_chunks)}

# ✅ STEP 4: Initialize Mistral client
api_key = "08nQ6qoukkoLaW0XZ1RX3RiEANjjUUjK"  # or use: api_key = "your-api-key"
model = "mistral-large-latest"  # or "mistral-small-latest"

client = Mistral(api_key=api_key)

# ✅ STEP 5: Define RAG Question Answering Function
def rag_ask_question(query, top_k=3):
    # Embed query
    query_embedding = embed_model.encode([query])[0]

    # Search FAISS
    D, I = faiss_index.search(np.array([query_embedding]), top_k)
    retrieved_chunks = [index_to_chunk[i] for i in I[0]]

    # Build prompt
    context = "\n---\n".join(retrieved_chunks)
    prompt = f"""You are a helpful assistant answering questions from a PDF document.
Use the context below to answer the question concisely and clearly.
If the query is about a table, visualize relevant tables.

If the context is insufficient, state that clearly instead of speculating.

### Context:
{context}

### Question:
{query}
"""

    # Ask Mistral
    response = client.chat.complete(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
    )
    return response.choices[0].message.content



🔍 Embedding document chunks...


In [ ]:
in markdown format (without truncation)

In [25]:
question = "explain Statement of changes in net assets for the year ended as at 31 March 2008"
answer = rag_ask_question(question)
print("\n🤖 Mistral RAG Answer:\n", answer)



🤖 Mistral RAG Answer:
 The Statement of Changes in Net Assets for the year ended as at 31 March 2008 provides a detailed breakdown of changes in various funds over the financial year. Here is a summary of the key components:

|                                          | Surplus funds R’000   | Capital funds R’000   | Deferred income R’000   | Total R’000   |
|:-----------------------------------------|:------------------------|:------------------------|:--------------------------|:--------------|
| Balance at 1 April 2006                  | 10 225                  | 75 050                  | 4 331                     | 89 606        |
| Net surplus for the year                 | 5 011                   | -                       | -                         | 5 011         |
| Transfer to surplus fund                 | 385                     | (385)                   | (3 100)                   | (3 100)       |
| Balance at 1 April 2007                  | 15 621                  | 74 